## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
data = list(range(1,1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [2]:
import pyspark

sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [3]:
rdd = sc.parallelize(data, 10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [4]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [5]:
# count
rdd.count()

1000

In [6]:
# first
rdd.first()

1

In [7]:
# take
rdd.take(23)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23]

In [8]:
# top
rdd.top(20)

[1000,
 999,
 998,
 997,
 996,
 995,
 994,
 993,
 992,
 991,
 990,
 989,
 988,
 987,
 986,
 985,
 984,
 983,
 982,
 981]

In [9]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [10]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([1.95275931e-01, 1.00656443e+00, 1.51162535e+00, 1.16768660e+00,
       3.03584972e+00, 7.24663759e-01, 4.99298660e+00, 3.18280268e+00,
       8.17380405e+00, 3.14037338e+00, 9.83385067e+00, 7.52149080e+00,
       4.55405646e-01, 2.74728992e+00, 1.51921308e+00, 1.53371248e+00,
       1.15309117e+01, 1.24261251e+01, 3.80094595e+00, 9.05862613e+00,
       1.21541341e+01, 6.49353949e+00, 1.33551143e+00, 1.04801389e+01,
       2.29590864e+01, 6.96393058e+00, 1.00299539e+01, 7.29309829e+00,
       1.43487825e+01, 1.32789061e+00, 1.77800443e+01, 1.91955392e+01,
       1.54511055e+01, 2.52935555e+01, 1.67131881e+01, 3.71379747e-01,
       1.59375152e+01, 1.21975179e+01, 2.29718546e+01, 9.38890054e+00,
       1.25719007e+01, 1.61469460e+01, 6.50597488e+00, 6.11321716e+00,
       3.04086111e+01, 3.82292943e+01, 3.23943669e+01, 1.87902166e+01,
       2.43511870e+01, 3.17532131e+01, 3.14910988e+01, 1.40902316e+01,
       2.49357852e+00, 1.16402538e+01, 1.35177154e+01, 4.28830812e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [11]:
price_items = sc.parallelize(sales_figures, 10)

price_items.top(20)

[950.3783716004813,
 935.307154936557,
 929.6879299849807,
 927.132510542634,
 923.23875394269,
 901.6486210789244,
 900.9153829020863,
 898.9305609237214,
 881.5671963121729,
 869.3202940504873,
 848.823371415127,
 844.9722314922088,
 842.355110997914,
 840.2117138386764,
 839.1507561376964,
 836.9995112188602,
 830.6051471674109,
 826.2581933199989,
 821.2406061353619,
 817.741523360367]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [12]:
def sales_tax(num):
    return num*0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [13]:
# perform action to retrieve rdd values
revenue_minus_tax.top(20)

[874.3481018724428,
 860.4825825416325,
 855.3128955861822,
 852.9619096992233,
 849.3796536272748,
 829.5167313926105,
 828.8421522699194,
 827.0161160498237,
 811.041820607199,
 799.7746705264483,
 780.9175017019169,
 777.3744529728322,
 774.966702118081,
 772.9947767315823,
 772.0186956466807,
 770.0395503213514,
 764.156735394018,
 760.157537854399,
 755.5413576445329,
 752.3222014915376]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [14]:
discounted = revenue_minus_tax.map(lambda item: item*0.9)

In [15]:
discounted.take(10)

[0.1616884705597379,
 0.8334353499148739,
 1.251625792871522,
 0.9668445086782532,
 2.513683568180188,
 0.6000215920549344,
 4.134192907489601,
 2.635360619237589,
 6.76790975320636,
 2.600229160359497]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [16]:
price_items.map(sales_tax).map(lambda item: item*0.9).top(15)

[786.9132916851986,
 774.4343242874693,
 769.781606027564,
 767.6657187293009,
 764.4416882645473,
 746.5650582533494,
 745.9579370429275,
 744.3145044448414,
 729.9376385464792,
 719.7972034738035,
 702.8257515317252,
 699.637007675549,
 697.4700319062729,
 695.6952990584241,
 694.8168260820127]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [17]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at readRDDFromFile at PythonRDD.scala:262 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [18]:
mapped = price_items.map(lambda item: (item, sales_tax(item)*0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.19527593062770277, 0.1616884705597379), (1.0065644322643403, 0.8334353499148739), (1.5116253537095674, 1.251625792871522), (1.1676866046838805, 0.9668445086782532), (3.0358497200243817, 2.513683568180188), (0.7246637585204521, 0.6000215920549344), (4.99298660324831, 4.134192907489601), (3.182802680238634, 2.635360619237589), (8.173804049766135, 6.76790975320636), (3.1403733820766866, 2.600229160359497)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [19]:
flat_mapped = price_items.flatMap(lambda item: (item, sales_tax(item)*0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.19527593062770277, 0.1616884705597379, 1.0065644322643403, 0.8334353499148739, 1.5116253537095674, 1.251625792871522, 1.1676866046838805, 0.9668445086782532, 3.0358497200243817, 2.513683568180188]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [20]:
# use the filter function
selected_items = discounted.filter(lambda item: item>300)

# calculate total remaining in inventory 
selected_items.count()

264

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [21]:
selected_items.reduce(lambda x,y: x+y)

122711.01286161321

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [22]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(39, 300.16611380225396),
 (47, 304.8384355823506),
 (17, 304.7948875956369),
 (46, 329.67064276811305),
 (27, 313.8536058121763),
 (5, 319.22042066279823),
 (42, 339.7639028614058)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [23]:
# calculate how much each user spent
# "To do this we can use a method called .reduceByKey() to perform reducing operations while grouping by keys."
rev_by_user = sales_data.reduceByKey(lambda x,y: x+y)
rev_by_user.top(20)

[(50, 1358.9290632132145),
 (49, 3300.2875480843913),
 (48, 724.7196112722501),
 (47, 1067.141197520864),
 (46, 4388.243781680044),
 (45, 1933.418250079973),
 (44, 3199.9530951255665),
 (43, 3774.800096353806),
 (42, 3041.003279683421),
 (41, 363.44996216830384),
 (40, 1869.1684721924623),
 (39, 2386.4164917789553),
 (38, 4079.786918398512),
 (37, 1256.4512775621924),
 (36, 1702.8926462264667),
 (35, 2882.6051489552774),
 (34, 2481.974230753277),
 (33, 1795.763261366066),
 (32, 3632.7275955183286),
 (31, 1773.2269731195174)]

In [24]:
# sort the users from highest to lowest spenders
# "use the .sortBy() method on the RDD to rank the users from the highest spending to the least spending"
rev_by_user.sortBy(lambda rbui: rbui[1], ascending=False).collect()

[(20, 6076.716330324354),
 (46, 4388.243781680044),
 (38, 4079.786918398512),
 (29, 4077.245076247109),
 (11, 3895.897303719583),
 (43, 3774.800096353806),
 (9, 3753.1805975996303),
 (2, 3664.8799041450575),
 (32, 3632.7275955183286),
 (13, 3352.1584980346715),
 (49, 3300.2875480843913),
 (12, 3260.1735318319347),
 (44, 3199.9530951255665),
 (10, 3158.865802642015),
 (8, 3062.19451954707),
 (42, 3041.003279683421),
 (24, 2891.2037249423497),
 (35, 2882.6051489552774),
 (18, 2742.8798952774314),
 (22, 2595.1084128056245),
 (34, 2481.974230753277),
 (15, 2472.542190474933),
 (16, 2458.806569081664),
 (30, 2399.629891700695),
 (39, 2386.4164917789553),
 (7, 2153.9574802415955),
 (19, 2147.525632234995),
 (21, 2069.547077826436),
 (26, 2017.5753459355221),
 (6, 1958.4129367718665),
 (45, 1933.418250079973),
 (40, 1869.1684721924623),
 (27, 1805.4084989581634),
 (33, 1795.763261366066),
 (23, 1780.9098095934735),
 (31, 1773.2269731195174),
 (25, 1761.6032371532058),
 (14, 1745.9813252679578

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [25]:
# determine how many items were bought per user
sorted(sales_data.countByKey().items(), key=lambda uni: uni[1], reverse=True) 

[(38, 13),
 (24, 10),
 (39, 9),
 (32, 9),
 (12, 8),
 (42, 8),
 (7, 8),
 (15, 8),
 (31, 7),
 (23, 7),
 (26, 7),
 (29, 7),
 (28, 7),
 (22, 7),
 (13, 7),
 (27, 6),
 (3, 6),
 (40, 6),
 (18, 6),
 (47, 6),
 (50, 6),
 (2, 6),
 (14, 6),
 (16, 6),
 (6, 5),
 (35, 5),
 (19, 5),
 (43, 5),
 (33, 5),
 (25, 5),
 (9, 4),
 (48, 4),
 (1, 4),
 (5, 4),
 (45, 4),
 (20, 4),
 (11, 4),
 (49, 3),
 (41, 3),
 (17, 3),
 (46, 3),
 (21, 3),
 (34, 3),
 (36, 3),
 (10, 2),
 (37, 2),
 (30, 2),
 (44, 1),
 (8, 1),
 (4, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
